# Lab-2-Introduce Phi-3.5 Vision

In [ ]:
! pip install opencv-python

In [ ]:
import cv2
import numpy as np

In [ ]:
def save_keyframes(video_path, output_folder):
    videoCapture = cv2.VideoCapture(video_path)
    success, frame = videoCapture.read()
    i = 0
    while success:
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        hist = cv2.calcHist([gray_frame], [0], None, [256], [0, 256])
        
        success, next_frame = videoCapture.read()
        if not success:
            break
        
        next_gray_frame = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)
        
        next_hist = cv2.calcHist([next_gray_frame], [0], None, [256], [0, 256])
        
        similarity = cv2.compareHist(hist, next_hist, cv2.HISTCMP_CORREL)
        
        if similarity < 0.9:
            i += 1
            cv2.imwrite(f"{output_folder}/keyframe_{i}.jpg", frame)
            print(f"Saved keyframe {i}")
        
        frame = next_frame

    videoCapture.release()

In [ ]:
save_keyframes('../examples/video/car.mp4', './output')

In [ ]:
from PIL import Image
import requests, base64

In [ ]:
images = [] 
placeholder = "" 
for i in range(1,4): 
    with open("../examples/output/keyframe_"+str(i)+".jpg", "rb") as f:

        images.append(Image.open("../examples/output/keyframe_"+str(i)+".jpg"))
        placeholder += f"<|image_{i}|>\n"
        # print(i)

In [ ]:
from notebook_utils import device_widget

device = device_widget(default="GPU", exclude=["NPU"])

device

In [ ]:
from ov_phi3_vision import OvPhi3Vision

In [ ]:
out_dir = "../model/phi-3.5-vision-128k-instruct-ov"

In [ ]:
model = OvPhi3Vision(out_dir, device.value)

In [ ]:
from transformers import AutoProcessor, TextStreamer

messages = [
    {"role": "user", "content":  placeholder+"Summarize the video."},
]

processor = AutoProcessor.from_pretrained(out_dir, trust_remote_code=True)

prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = processor(prompt, images, return_tensors="pt")

generation_args = {"max_new_tokens": 500, "do_sample": False, "streamer": TextStreamer(processor.tokenizer, skip_prompt=True, skip_special_tokens=True)}

print("Summary:")
generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args)